# Quick start

Strategy similar to strategies on the legacy version of Quentiacs

* [User account](https://quantiacs.com/personalpage/homepage)
* [Documentation](https://quantiacs.com/documentation/en/)


> **Strategy idea.** We will open positions in futures where two moving averages of the closing prices intersect


> The trading algorithm uses financial data to form the weights, in proportion to which the capital is distributed. A positive weight means a long position (buy), a negative value means a short position (sell).

> For each date, the algorithm calculates what portfolio weights should be at the opening of the next day's trading.

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) { return false; }
// disable widget scrolling

<IPython.core.display.Javascript object>

In [2]:
import xarray as xr

import qnt.ta as qnta
import qnt.backtester as qnbt
import qnt.data as qndata


def load_data(period):
    return qndata.futures_load_data(tail=period)


def strategy(data):
    close = data.sel(field='close')
    sma_slow = close.rolling(time=200).mean().isel(time=-1) # qnta.sma(close, 200).isel(time=-1)
    sma_fast = close.rolling(time=20).mean().isel(time=-1) # qnta.sma(close, 20).isel(time=-1)
    return xr.where(sma_slow < sma_fast, 1, -1)


weights = qnbt.backtest(
    competition_type="futures",
    load_data=load_data,
    lookback_period=365,
    start_date='2006-01-01',
    strategy=strategy
)

Run last pass...
Load data...


100% (33771540 of 33771540) |############| Elapsed Time: 0:00:00 Time:  0:00:00


Run pass...
Ok.
---
Run first pass...
Load data...
Run pass...
Ok.
---
Load full data...
---
Run iterations...



100% (3894 of 3894) |####################| Elapsed Time: 0:00:18 Time:  0:00:18


Merge outputs...
Load data for cleanup and analysis...
Check missed dates...
Ok.
Normalization...
Done.
Write output: /root/fractions.nc.gz
---
Analyze results...
Check...
Check missed dates...
Ok.
Check sharpe ratio.


ERROR! The sharpe ratio is too low. -0.35782078850744853 < 1


Check correlation.



WARNING! This strategy correlates with other strategies.


The number of systems with a larger Sharpe ratio and correlation larger than 0.8: 3
The max correlation value (with systems with a larger Sharpe ratio): 0.9999962748007087
Current sharpe ratio(5y): -0.7367873282614694

---
Calc global stats...
---
Calc stats per asset...
Build plots...
---
Select the asset (or leave blank to display the overall stats):


interactive(children=(Combobox(value='', description='asset', options=('', 'F_AD', 'F_AE', 'F_AH', 'F_AX', 'F_…

## Implementing an identical strategy that works quickly


```python
import xarray as xr

import qnt.ta as qnta
import qnt.data as qndata
import qnt.output as qnout
import qnt.stats as qns

data = qndata.futures_load_data(min_date='2004-01-01',)

close = data.sel(field='close')
close = data.sel(field='close')
sma_slow = close.rolling(time=200).mean() # qnta.sma(close, 200)
sma_fast = close.rolling(time=20).mean() # qnta.sma(close, 20)
weights = xr.where(sma_slow < sma_fast, 1, -1)

weights = qnout.clean(weights, data)

qnout.check(weights, data)

qnout.write(weights)

# calc stats
stats = qns.calc_stat(data, weights.sel(time=slice('2006-01-01',None)))
stats.to_pandas().tail()
```

## if it is necessary to use several data sets in the strategy

```python
import xarray as xr
import numpy as np

import qnt.ta as qnta
import qnt.backtester as qnbt
import qnt.data as qndata


def load_data(period):
    futures = qndata.futures.load_data(tail=period, assets=['F_DX']).isel(asset=0)
    crypto = qndata.cryptofutures.load_data(tail=period)
    return {"futures": futures, "crypto": crypto}, futures.time.values


def window(data, max_date: np.datetime64, lookback_period: int):
    min_date = max_date - np.timedelta64(lookback_period, 'D')
    return {
        "futures": data['futures'].sel(time=slice(min_date, max_date)),
        "crypto": data['crypto'].sel(time=slice(min_date, max_date)),
    }


def strategy(data):
    close_futures = data['futures'].sel(field='close')
    close_crypto = data['crypto'].sel(field='close')
    # .isel(time=-1) get the last day
    sma200 = qnta.sma(close_futures, 20).isel(time=-1)
    sma200_crypto = qnta.sma(close_crypto, 20).isel(time=-1)
    return xr.where(sma200 < sma200_crypto, 1, -1)


qnbt.backtest(
    competition_type="cryptofutures",
    load_data=load_data,
    lookback_period=365,
    start_date='2014-01-01',
    strategy=strategy,
    window=window
)
```

# Guide with the most frequently required functions

## What libraries are there?

```python
# Import basic libraries.
import xarray as xr
import numpy as np
import pandas as pd

# Import quantnet libraries.
import qnt.data    as qndata
import qnt.output as output
import qnt.backtester as qnbt
import qnt.stats   as qnstats
import qnt.graph   as qngraph
import qnt.ta      as qnta # indicators library
import qnt.exposure as qne
```

## How to load data?

```python
data = qndata.futures.load_data(tail = 15*365, dims = ("time", "field", "asset"))
```

```python
data = qndata.cryptofutures.load_data(tail = 15*365, dims = ("time", "field", "asset"))
```

```python
data = qndata.stocks.load_data(tail = 15*365, dims = ("time", "field", "asset"))
```

```python
data = qndata.crypto.load_data(tail = 15*365, dims = ("time", "field", "asset"))
```

## How to view a list of all tickers?

```python
data.asset.to_pandas().to_list()
```

## What market data is there?

```python
data.field.to_pandas().to_list()
```

## How to unload a specific ticker?

```python
data = qndata.futures.load_data(tail=15 * 365, assets=['F_O', 'F_DX', 'F_GC'])
```

## How to use specific tickers?

```python
def get_data_filter(data, assets):
    filler = data.sel(asset=assets)
    return filler

get_data_filter(data, ['F_O', 'F_DX', 'F_GC'])
```

## How to get the prices for the previous day?

```python
qnta.shift(data.sel(field="open"), periods=1)
```

or

```python
data.sel(field="open").shift(time=1)
```

## How do I get a list of the top 10 Sharpe assets?

```python
import qnt.stats as qnstats

data = qndata.futures.load_data(tail=16 * 365)

def get_best_instruments(data, weights, top_size):
    top_period = 3 * 365
    stats_per_asset = qnstats.calc_stat(data, weights, per_asset=True)
    # calculate ranks of assets by "sharpe_ratio"
    ranks = (-stats_per_asset.sel(field='sharpe_ratio')).rank('asset')
    # Select top assets by rank which assets have 'top_period' days ago.
    # We assume, that the "sharpe_ratio" of these assets will be good in the next 'top_period' days
    rank = ranks.isel(time=-top_period)
    top = rank.where(rank <= top_size).dropna('asset').asset

    # select top stats
    top_stats = stats_per_asset.sel(asset=top.values)

    # print results
    print("SR tail of the top assets:")
    display(top_stats.sel(field='sharpe_ratio').to_pandas().tail())

    print("avg SR = ", top_stats[-top_period:].sel(field='sharpe_ratio').mean('asset')[-1].item())
    display(top_stats)
    return top_stats.coords['asset'].values

get_best_instruments(data, weights, 10)
```

## How can I check the results for only the top 10 Sharpe assets?

Download the Top 10 Assets and apply your strategy to it

```python
best_assets = get_best_instruments(data, weights, 10)

data = qndata.futures.load_data(tail=15 * 365, assets=best_assets)
...
```

## How can prices be processed?

Example logarithm

```python
import numpy as np

high = np.log(data.sel(field="high"))
```

## How can you reduce the commission when trading?

```python
def get_lower_commissions(weights, rolling_time=6):
    return weights.rolling({"time": rolling_time}).max()

improved_weights = get_lower_commissions(weights, rolling_time=6)
```

## How to use technical analysis indicators?

for available indicators see the source code of the library (/qnt/ta)

## ATR

```python
def get_atr(data, days=14):
    high = data.sel(field='high') * 1.0 
    low = data.sel(field='low') * 1.0 
    close = data.sel(field='close') * 1.0

    return qnta.atr(high, low, close, days)

atr = get_atr(data, days=14)
```

## EMA

```python
prices = data.sel(field="high")
prices_ema = qnta.ema(prices, 15)
```

## TRIX

```python
prices = data.sel(field="high")
prices_trix = qnta.trix(prices, 15)
```

## ADL and EMA

```python
adl = qnta.ad_line(data.sel(field="close")) * 1.0 
adl_ema = qnta.ema(adl, 18)
```

## How do you know if your strategy is good?

```python
qnout.check(weights, data)
```

or

```python
stat = qnstats.calc_stat(data, weights)
display(stat.to_pandas().tail())
```

or

```python
import qnt.graph   as qngraph
statistics = qnstats.calc_stat(data, weights)
display(statistics.to_pandas().tail())

performance = statistics.to_pandas()["equity"]
qngraph.make_plot_filled(performance.index, performance, name="PnL (Equity)", type="log")

display(statistics[-1:].sel(field = ["sharpe_ratio"]).transpose().to_pandas())
qnstats.print_correlation(weights, data)
qnstats.check_exposure(weights)

```

or

```python
import qnt.stats   as qnstats

def estimate_sharpe(data, weights_final):
    stat = qnstats.calc_stat(data, weights_final)
    days = len(stat.coords["time"])
    returns = stat.loc[:, "relative_return"]
    
    sharpe_ratio = qnstats.calc_sharpe_ratio_annualized(
        returns,
        max_periods=days,
        min_periods=days).to_pandas().values[-1]
    
    print(f'Sharpe ratio = {sharpe_ratio}')

estimate_sharpe(data, weights)
```

## An example implementation using pandas series

```python
def get_price_pct_change(prices):
    prices_pandas = prices.to_pandas()
    assets = data.coords['asset'].values
    for asset in assets:
        prices_pandas[asset] = prices_pandas[asset].pct_change()
    return prices_pandas


prices = data.sel(field='close') * 1.0
prices_pct_change = get_price_pct_change(prices).unstack().to_xarray()

```

## How to submit a strategy to the competition?

Check that there is a check in the code

```python
qnout.check(weights, data)
```

if everything is ok, write the weights to the file

```python
output.write(weights)
```

In your **personal account**:

* **choose** a strategy;
* click on the "**Submit**" button;
* select the type of competition.

Further, the strategy will go to the сheck (Competition > Checking)
